In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

data = pd.read_csv("Titanic - Machine Learning From a Disaster/train.csv")

print(data.head())


   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [2]:
#looking at the dataset

data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#transform column Sex into numerical variables

def transform(data):
    data.Sex.replace(['male', 'female'],
                        [0, 1], inplace=True)

transform(data)


In [4]:
data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S


In [5]:
#split data into training and validation set

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

X = data[["Pclass","Sex","Age","SibSp","Parch","Fare"]]
y = data[["Survived"]]    

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [6]:
#dealing with missing values

from sklearn.impute import SimpleImputer
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

for col in cols_with_missing:
    X_train[col + '_was_missing'] = X_train[col].isnull()
    X_valid[col + '_was_missing'] = X_valid[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

In [7]:
#dropping unnecessary columns

imputed_X_valid.head()
imputed_X_valid.drop("Age_was_missing",axis=1,inplace=True)


In [8]:
#create and fit the model

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error

model =DecisionTreeClassifier()
model.fit(imputed_X_train,y_train)
titanic_preds =model.predict(imputed_X_valid)
print(mean_absolute_error(y_valid,titanic_preds))

0.20670391061452514


In [9]:
#reading other dataset

xtest = pd.read_csv("Titanic - Machine Learning From a Disaster/test.csv")
xtest.head()
transform(xtest)
npheader = xtest["PassengerId"]

In [10]:
xtest.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [11]:
xtest2 = xtest[["Pclass","Sex","Age","SibSp","Parch","Fare"]]

cols_with_missing = [col for col in xtest2.columns
                     if xtest2[col].isnull().any()]

for col in cols_with_missing:
    xtest2[col + '_was_missing'] = xtest2[col].isnull()
    

# Imputation
my_imputer = SimpleImputer()
imputed_xtest2 = pd.DataFrame(my_imputer.fit_transform(xtest2))

# Imputation removed column names; put them back
imputed_xtest2.columns = xtest2.columns

C:\Users\danie\AppData\Local\Temp\ipykernel_1256\4039880484.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest2[col + '_was_missing'] = xtest2[col].isnull()
C:\Users\danie\AppData\Local\Temp\ipykernel_1256\4039880484.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest2[col + '_was_missing'] = xtest2[col].isnull()


In [12]:
imputed_xtest2.head()
imputed_xtest2.drop("Fare_was_missing",axis=1,inplace=True)
imputed_xtest2.drop("Age_was_missing",axis=1,inplace=True)

In [13]:
imputed_X_valid.head()

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3.0,0.0,29.745184,0.0,0.0,14.4583
1,3.0,0.0,29.745184,0.0,0.0,7.5500
2,3.0,0.0,7.000000,4.0,1.0,29.1250
3,1.0,1.0,29.745184,1.0,0.0,146.5208
4,3.0,1.0,29.000000,0.0,2.0,15.2458


In [14]:
#predicting results

results= model.predict(imputed_xtest2)
print(results)

[0 0 0 1 0 1 0 0 1 0 0 0 1 0 1 0 0 1 1 0 0 1 1 0 1 0 1 1 1 0 0 0 1 0 1 0 0
 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 1 0 1 1 1 0 0 0 1 0 0 0 1
 1 0 0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 1 1 1 0 1 0 0 0 1 0 0 0 0 0 0
 0 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 0 1 1 0 0 1 0 0 1 0 0 1 1 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 0 1 1 1 0 0 0 0 1 0 1 0 0 0 0 1 0 1 1 0 1 0
 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 1 1 1 1 1 1 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0
 1 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 1 1 1 0 1 1 1
 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 1 0 1 0
 1 0 1 0 1 1 0 1 0 0 0]


In [15]:
print(results.shape)
print(npheader.shape)

(418,)
(418,)


In [18]:
predictions= pd.DataFrame({"PassengerId":npheader,"Survived":results}, columns = ['PassengerId','Survived'])

In [19]:
predictions.to_csv('Titanic - Machine Learning From a Disaster/predictions.csv',index=False)